In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

import gym
import trading_env

In [2]:
env = gym.make('trading-v0')
env_trading = gym.make('test_trading-v0')

In [21]:
print(env_trading.observation_space.contains(np.array([a]))) #why?
print(env_trading.action_space.sample()[0])

here
False
0.185761
